In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import cv2


In [2]:
# Set some paths
# Download images to a folder in this directory and set PATH variable to its path
PATH = "/Users/jeremystubbs/Desktop/Python/Machine_Learning_NNs/Mobilenet/custom/asl_images/"
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'valid')



In [3]:
# Make training and validation Datasets
BATCH_SIZE = 10
IMG_SIZE = (224, 224)

train_dataset = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(directory=train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE)
validation_dataset = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(directory=validation_dir,target_size=IMG_SIZE, batch_size=BATCH_SIZE)

# train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
#                                                             labels = "inferred",
#                                                             shuffle=True,
#                                                             batch_size=BATCH_SIZE,
#                                                             image_size=IMG_SIZE)

# validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
#                                                                 labels = "inferred",
#                                                                 shuffle=True,
#                                                                 batch_size=BATCH_SIZE,
#                                                                 image_size=IMG_SIZE)


Found 22655 images belonging to 24 classes.
Found 4800 images belonging to 24 classes.


In [ ]:
# Plot some images - only works with keras utils dataset not flow from directory
class_names = train_dataset.class_names
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [4]:
# Define preprocessing function and create train and validation iterators

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

# This does the same thing
# rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

# Could include preprocessing here
# train_dataset = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(directory=train_dir, target_size=(224,224), batch_size=10)
# validation_dataset = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(directory=validation_dir,target_size=(224,224), batch_size=10)


# image_batch, label_batch = next(iter(train_dataset))


In [5]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False



In [17]:
##This cell creates the model
num_of_classes = 24

# Create custom layers
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
# units here equals number of classes
prediction_layer = tf.keras.layers.Dense(units=num_of_classes, activation='softmax')
identity_layer = tf.keras.layers.Layer()

inputs = tf.keras.Input(shape=(224, 224, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = identity_layer(x)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

# This stuff was from google - not really applicable here
# Add input, preprocessing, global average and prediction layer to the base model
# feature_batch, and prediction_batch are optional 
# feature_batch = base_model(image_batch)
# feature_batch_average = global_average_layer(feature_batch)
# prediction_layer = tf.keras.layers.Dense(len(class_names),
                        #   kernel_regularizer=tf.keras.regularizers.l2(0.0001))
# prediction_batch = prediction_layer(feature_batch_average)


In [18]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 layer (Layer)               (None, 7, 7, 1280)        0         
                                                             

In [19]:
##Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Here is an alternative setup
# model.compile(
#   optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9), 
#   loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
#   metrics=['accuracy'])

# model.summary()



In [20]:
#Train the model
initial_epochs = 5

model.fit(train_dataset,
                    epochs=initial_epochs, 
                    validation_data=validation_dataset, )



2022-05-07 15:15:23.710404: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/5
2266/2266 [==============================] - 222s 98ms/step - loss: 1.1292 - accuracy: 0.7311 - val_loss: 0.3310 - val_accuracy: 0.9621
Epoch 2/5
2266/2266 [==============================] - 222s 98ms/step - loss: 0.2497 - accuracy: 0.9636 - val_loss: 0.1303 - val_accuracy: 0.9881
Epoch 3/5
2266/2266 [==============================] - 222s 98ms/step - loss: 0.1168 - accuracy: 0.9875 - val_loss: 0.0672 - val_accuracy: 0.9960
Epoch 4/5
1602/2266 [====================>.........] - ETA: 53s - loss: 0.0688 - accuracy: 0.9946

KeyboardInterrupt: 

In [52]:
# Optional fine-tuning I read on tensorflow's official website
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# # Fine-tune from this layer onwards
fine_tune_at = 100

base_model.trainable = True
# # Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

Number of layers in the base model:  154


In [54]:
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 tf.math.truediv_9 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_9 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 layer_4 (Layer)             (None, 7, 7, 1280)        0         
                                                           

In [55]:
##Recompile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])


In [57]:
#Train the model
initial_epochs = 10

model.fit(train_dataset,
                    epochs=initial_epochs, 
                    validation_data=validation_dataset, )

Epoch 1/10
  5/372 [..............................] - ETA: 3:25 - loss: 0.0705 - accuracy: 0.9800

KeyboardInterrupt: 

In [21]:
##Save model
os.chdir("/Users/jeremystubbs/Desktop/Python/Machine_Learning_NNs/Mobilenet/custom/")
!mkdir -p saved_model_asl
model.save('saved_model_asl/my_model')

2022-05-07 15:41:14.281742: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: saved_model_asl/my_model/assets


/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.10/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [37]:
## Predict images in test set with model 
label_map = ["A","B","C","D","E","F","G","H","I","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y"]

threshold = 0.25

img = cv2.imread("test_images/A_test.jpg")
resized = cv2.resize(img, (224, 224))
resized = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
resized = cv2.cvtColor(resized, cv2.COLOR_GRAY2RGB)

reshaped = resized.reshape(1, 224, 224, 3)

predictions = model.predict(reshaped, verbose=0)

for val in predictions[0]:
    if val > threshold:
        index_of = np.where(predictions[0]==val)
        print(label_map[index_of[0][0]])
print(predictions)


A
[[2.7210033e-01 1.3896777e-01 1.2281845e-02 2.7797181e-05 3.1982403e-02
  4.9487013e-02 9.4313323e-03 4.0548993e-03 7.8144958e-03 2.2910314e-03
  2.4812387e-03 1.1629248e-02 1.7272384e-03 2.8669236e-03 4.3006893e-03
  7.7096703e-03 1.6573543e-02 2.2908860e-04 2.1255569e-01 6.4689092e-02
  7.1655908e-03 1.2905003e-01 9.9131307e-03 6.6991302e-04]]
